In [1]:
!pip install -U whisper openai ffmpeg pillow opencv-python
!sudo apt-get install libimage-exiftool-perl
!sudo apt-get install ffmpeg
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip install openai-clip
!pip install opencv-python
!pip install summa
!pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=fad68fde12d080bc33bffcebfd07effea8b2e150d0a33750f0e565cff208d107
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=9c85c02aafe324cc80628fca7b09e9439e3e66d53085f80ce25a940e4a18623f
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built whisper ffmpeg
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-zip-perl libmime-charset-perl libsombok3
  libunicode-linebreak-perl
Suggested packages:
  libposix-strptime-perl libencode-han

In [24]:
import whisper
import subprocess
import cv2
from PIL import Image
import os
import numpy as np
from transformers import BlipProcessor, BlipForConditionalGeneration
import json
import argostranslate.package
import argostranslate.translate
from argostranslate.translate import translate


def contains_english(text):
    return any("a" <= c.lower() <= "z" for c in text)


def get_en_kr_model():
    available_packages = argostranslate.package.get_available_packages()

    package_to_install = list(
        filter(lambda x: x.from_code == "en" and x.to_code == "ko", available_packages)
    )[0]

    download_path = package_to_install.download()
    argostranslate.package.install_from_path(download_path)


# 영상에서 메타데이터 추출 (Exiftool 사용)
def get_metadata(filepath):
    cmd = ["exiftool", "-j", filepath]
    result = subprocess.run(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    if result.returncode != 0:
        print("Error:", result.stderr)
        return None

    find_key = ["createdate", "gpslatitude", "gpslongitude"]
    metadata = json.loads(result.stdout)[0]
    text_metadata = {}
    for key, value in metadata.items():
        for find in find_key:
            if (find in key.lower()) and (
                (
                    (find == find_key[0])
                    and (
                        ((find in text_metadata) and (text_metadata[find] < value))
                        or (not (find in text_metadata))
                    )
                )
                or (find != find_key[0])
            ):
                text_metadata[find] = value
                break
    return text_metadata


# 영상에서 오디오 추출 (FFmpeg 사용)
def extract_audio(video_path, audio_path="audio.wav"):
    command = ["ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a", audio_path, "-y"]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return audio_path


# Whisper를 사용하여 오디오를 텍스트로 변환
def transcribe_audio(audio_path, model):
    result = model.transcribe(audio_path)
    return result["text"]


# 영상 프레임 추출 (1초당 1프레임)
def extract_frames(video_path, output_folder="frames", frame_rate=0.2):
    os.makedirs(output_folder, exist_ok=True)
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vf",
        f"fps={frame_rate}",
        f"{output_folder}/frame_%04d.png",
        "-y",
    ]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


# BLIP 이미지 설명 생성
def generate_caption(image_path, processor, model):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    output = model.generate(**inputs)
    return processor.decode(output[0], skip_special_tokens=True)


def make_text_display_in_video(processor, model, frame_folder="frames"):
    text = []

    for i in range(1, 1000):
        frame_path = f"{frame_folder}/frame_{i:04d}.png"
        try:
            caption = generate_caption(frame_path, processor, model)
            caption = translate(caption, "en", "ko")
            words = caption.split(" ")
            kor_words = []
            for j in range(len(words)):
                if not contains_english(words[j]):
                    kor_words.append(words[j])
            caption = " ".join(kor_words)
            text.append(caption)
        except FileNotFoundError:
            break

    return "\n".join(text)


def process_video(video_file, audio_model, processor, video_model):
    print("Step 1: 영상에서 메타 데이타 추출 중...")
    text_metadata = get_metadata(video_file)

    print("Step 2: 영상에서 오디오 추출 중...")
    audio_file = extract_audio(video_file)

    print("Step 3: Whisper로 음성 변환 중...")
    text_audio = transcribe_audio(audio_file, audio_model)

    print("Step 4: 프레임 추출 중...")
    extract_frames(video_file)

    print("Step 5: 프레임 설명 생성 중...")
    text_display = make_text_display_in_video(processor, video_model)

    # 최종 결과 저장
    print(text_metadata)
    print(text_audio)
    print(text_display)

    subprocess.run(
        ["rm", "-rf", "frames", "audio.wav"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )


get_en_kr_model()
audio_model = whisper.load_model("base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
video_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)
video_file = "test.mp4"
process_video(video_file, audio_model, processor, video_model)

Step 1: 영상에서 메타 데이타 추출 중...
Step 2: 영상에서 오디오 추출 중...
Step 3: Whisper로 음성 변환 중...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Step 4: 프레임 추출 중...
Step 5: 프레임 설명 생성 중...
{'createdate': '2015:07:09 02:33:33'}
 송강 정철이 사랑한 고장, 강원도 고성. 깊은 역사와 문화가 우리의 가슴을 뛰게 합니다. 보고 듣고 채워 마며 깊이 있는 전통의 향기를 느껴봅니다. 내 인생의 잊지 못할 영혼의 장소. 고성으로의 여행을 떠나보세요. 내 인생의 잊지 못할 영혼의 장소.
숲과 산을 배경으로
그것에 쓰기를 가진 종이의 조각
울타리 지역에 말을 타고 사람
집 앞에 서있는 사람
불 덩굴에 앉아있는 사람들의 그룹
바다의 전망
